In [ ]:
%load_ext autoreload
%autoreload 2
import glob, sys
import numpy as np
import matplotlib.pyplot as plt
import time
import pyccl as ccl
def binning(corner): return [[corner[i],corner[i+1]] for i in range(len(corner)-1)]
#define cosmo
cosmo = ccl.Cosmology(Omega_c = 0.30711 - 0.048254, Omega_b = 0.048254, h = 0.677, sigma8 = 0.8288, n_s=0.96)

In [ ]:
%load_ext autoreload
%autoreload 2
import CL_COUNT_class_cluster_abundance as cl_count
import CL_COUNT_modeling_richness as richness_model

# Define proxy modelling
Use a mass proxy, define the probability for observing a proxy given a mass and redhsift

$$
P(\log\lambda|M,z) = N(\mu(M,z), \sigma^2(M,z))
$$
the mean is
$$
\mu(M,z) = \mu_0 + a_\mu^M\log_{10}\frac{M}{M_0} + a_M^z\log_{10}\frac{1+z}{1+z_0} 
$$
variance is
$$
\sigma(M,z) = \sigma_0 + a_\sigma^M\log_{10}\frac{M}{M_0} + a_\sigma ^z\log_{10}\frac{1+z}{1+z_0} 
$$

In [ ]:
#M_0, z_0
theta_pivot = [3e14/0.71, 0.6]
#\mu_0, a_\mu^z, a_\mu^M
theta_mu = [3.19, -0.7, 2]
#\sigma_0, a_\sigma^z, a_\sigma^M
theta_sigma = [0.33, 0.,-0.08]
#Richness object
richness = richness_model.Richness(theta_mu, theta_sigma, theta_pivot)

In [ ]:
plt.figure(figsize = (7,5))
proxy_array = np.linspace(0,10,1000)
logm_array = np.linspace(14, 17, 4)
c = ['r', 'k', 'g', 'b']
for i, logm_ in enumerate(logm_array):
    p = richness.pdf(proxy_array, .0, logm_)
    plt.plot(proxy_array, p, label=f'logm = {logm_:.2f}', c = c[i])
    p = richness.pdf(proxy_array, .5, logm_,)
    plt.plot(proxy_array, p, '--', c = c[i])
plt.ylabel(r'$P(\lambda|\log M, z)$', fontsize = 20)
plt.ylim(0,3)
plt.legend()
plt.xlabel('$\ln \lambda$', fontsize = 20)
plt.text(0, 1.5,'full line: z=0', fontsize = 20)
plt.text(0, 2,'full line: z=.5', fontsize = 20)
plt.show()

In [ ]:
plt.figure(figsize = (7,5))
z_array = np.linspace(0,2,4)
logm_array = np.linspace(13, 15, 300)
c = ['r', 'k', 'g', 'b']
for i, z in enumerate(z_array):
    p = richness.mu_loglambda_logM_f(z, logm_array)
    plt.plot(logm_array, p, label=f'z = {z:.2f}', c = c[i])
plt.ylabel(r'$\langle\log\lambda|\log M, z\rangle$', fontsize = 20)
plt.legend()
plt.xlabel('$\log_{10}M$', fontsize = 20)
plt.show()

initialize the `ClusterAbundance` object

In [ ]:
clc = cl_count.ClusterAbundance()
#choose mass function and mass definition
massdef = ccl.halos.massdef.MassDef(200, 'critical', c_m_relation=None)
hmd = ccl.halos.hmfunc.MassFuncTinker08(cosmo, mass_def=massdef)
clc.set_cosmology(cosmo = cosmo, hmd = hmd, massdef = massdef)
clc.sky_area = (0.25)*4*np.pi

# Un-binned approach

$$
x_i = \int_{\log_{10}M_{min}}^{\log_{10}M_{min}}\frac{d^2V(z_i)}{dz d\Omega}\frac{dn(M,z_i)}{d\log_{10}M}P(\lambda_i|M,z_i)d\log_{10}M
$$

In [ ]:
import random
n = 100
z_list = np.linspace(0.1,2,n)
random.shuffle(z_list)
proxy_list = np.linspace(1,4,n)

pre tabulated grids to fasten integration

In [ ]:
z_grid = np.linspace(0., 3, 1000)
logm_grid = np.linspace(12,16, 1000)
clc.compute_pdf_grid_ProxyZ(proxy = proxy_list, z = z_list, proxy_model = richness, 
                            z_grid = z_grid, logm_grid = logm_grid)

Three different methods:
1. Simpson integration of a tabulated grid of $\frac{d^2V}{dz d\Omega}\frac{dn}{d\log_{10}M}P(\lambda|M,z)$ (method `simps`)
2. Double integral using `scipy.integrate.dblquad` of $\frac{d^2V}{dz d\Omega}\frac{dn}{d\log_{10}M}P(\lambda|M,z)$ (method `exact_CCL`)

`simps`

In [ ]:
time0 = time.time()
clc.compute_multiplicity_grid_MZ(z_grid = z_grid, logm_grid = logm_grid)
dndproxy_simps = clc.multiplicity_function_individual_ProxyZ(z = z_list, proxy = proxy_list, 
                                                             proxy_model = richness, method = 'simps')
time1 = time.time()
print(time1-time0)

`exact_CCL`

In [ ]:
time0 = time.time()
dndproxy_exact_CCL = clc.multiplicity_function_individual_ProxyZ(z = z_list, proxy = proxy_list, 
                                                                 proxy_model = richness, method = 'exact_CCL')
time1 = time.time()
print(time1-time0)

In [ ]:
plt.figure(figsize = (7,5))
plt.scatter(z_list, 100*abs(np.array(dndproxy_exact_CCL)-np.array(dndproxy_simps))/np.array(dndproxy_exact_CCL), s = 7, c = proxy_list, cmap = 'viridis')
plt.colorbar()
plt.xlabel('redshift', fontsize = 20)
plt.ylabel('|bias| to CCL $\%$', fontsize = 20)
plt.yscale('log')
plt.grid(True, which="both")

In [ ]:
%load_ext autoreload
%autoreload 2
import CL_COUNT_class_cluster_abundance as cl_count
import CL_COUNT_modeling_richness as richness_model

In [ ]:
clc = cl_count.ClusterAbundance()
#choose mass function and mass definition
massdef = ccl.halos.massdef.MassDef(200, 'critical', c_m_relation=None)
hmd = ccl.halos.hmfunc.MassFuncTinker08(cosmo, mass_def=massdef)
clc.set_cosmology(cosmo = cosmo, hmd = hmd, massdef = massdef)
clc.sky_area = (0.25)*4*np.pi

# Binned approach

Comptue the integral

$$
N_{\alpha\beta}^{\rm predicted} = \Omega_s\int_{z_\alpha}^{z_{\alpha + 1}} dz\int_{\lambda_{\beta}}^{\lambda_{\beta + 1}}d\lambda\int_{\log_{10}M_{min}}^{\log_{10}M_{max}}\frac{d^2V(z)}{dz d\Omega}\frac{dn(M,z)}{d\log_{10}M}P(\lambda|M,z)d\log_{10}M
$$

define binning

In [ ]:
z_corner = np.linspace(0.25, 1, 3)
proxy_corner = np.linspace(0, 2, 2)
Z_bin = binning(z_corner)
Proxy_bin = binning(proxy_corner)

pre tabulated grid (only depends on mass-proxy relation)

In [ ]:
clc.compute_cumulative_grid_ProxyZ(Proxy_bin = Proxy_bin, proxy_model = richness, 
                                   z_grid = z_grid, logm_grid = logm_grid)

Three different methods:
1. 2D Simpson integration on M-z axis of a tabulated grid of $\Omega\frac{d^2V(z)}{dz d\Omega}\frac{dn(M,z)}{d\log_{10}M}\int_{\lambda_{\rm bin}}P(\lambda|M,z)d\lambda$ (method `simps`)
2. Double integral using `scipy.integrate.dblquad` over a interpolated function based on a tabulated grid of $\Omega\frac{d^2V(z)}{dz d\Omega}\frac{dn(M,z)}{d\log_{10}M}\int_{\lambda_{\rm bin}}P(\lambda|M,z)d\lambda$ (method `dblquad_interp`)
3. Double integral using `scipy.integrate.dblquad` of $\Omega\frac{d^2V(z)}{dz d\Omega}\frac{dn(M,z)}{d\log_{10}M}\int_{bin_i} d\lambda P(\lambda|M,z)d\lambda$ (method `exact_CCL`

`simps`

In [ ]:
time0 = time.time()
clc.compute_multiplicity_grid_MZ(z_grid = z_grid, logm_grid = logm_grid)
N_ProxyZ_simps = clc.Cluster_Abundance_ProxyZ(Redshift_bin = Z_bin, Proxy_bin = Proxy_bin, 
                                              logm_limit = [min(logm_grid), max(logm_grid)], 
                                 proxy_model = richness, method = 'simps')
time1 = time.time()
print('time (seconds) = ' + str(time1-time0))

`dblquad_interp`

In [ ]:
time0 = time.time()
N_ProxyZ_dblquad_interp = clc.Cluster_Abundance_ProxyZ(Redshift_bin = Z_bin, Proxy_bin = Proxy_bin, 
                                                       logm_limit = [min(logm_grid), max(logm_grid)], 
                                 proxy_model = richness, method = 'dblquad_interp')
time1 = time.time()
print('time (seconds) = ' + str(time1-time0))

`exact_CCL`

In [ ]:
time0 = time.time()
N_ProxyZ_exact_CCL = clc.Cluster_Abundance_ProxyZ(Redshift_bin = Z_bin, Proxy_bin = Proxy_bin, 
                                                  logm_limit = [min(logm_grid), max(logm_grid)], 
                                 proxy_model = richness, method = 'exact_CCL')
time1 = time.time()
print('time (seconds) = ' + str(time1-time0))

In [ ]:
plt.figure(figsize = (7,5))
plt.yscale('log')
z_ = np.mean(Z_bin, axis = 1)
mass = np.mean(Proxy_bin, axis = 1)
plt.tick_params(axis='both', which="both", labelsize= 15)
for f in range(len(Proxy_bin)):
    label = [None, None]
    if f == 0:
        label = ['simps', 'dblquad_interp']
    y_simps = abs(100*(1-N_ProxyZ_simps[:,f]/N_ProxyZ_exact_CCL[:,f]))
    y_dbquad_interp = abs(100*(1-N_ProxyZ_dblquad_interp[:,f]/N_ProxyZ_exact_CCL[:,f]))
    plt.scatter(z_, y_simps, color = 'k', label = label[0])
    plt.scatter(z_, y_dbquad_interp, color = 'r', label = label[1])
plt.grid(True, which="both" )
plt.ylabel(r'|biais| to exact_CCL $(\%)$', fontsize = 20)
plt.xlabel(r'$redshift$', fontsize = 20)
plt.legend(frameon = False, fontsize = 20)